# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15101 MiB


https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth

## Install Requirements

In [ ]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.

## Imports

In [ ]:
from natsort import natsorted
from glob import glob
from os import sep
from os.path import exists, join

import io
import imageio

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

import numpy as np

from tqdm.notebook import tqdm

import gradio as gr

from functools import partialmethod

In [ ]:
DATA_DIR = '../datasets/generated_datasets'

if exists("/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion"):
  WORK_DIR = "/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion"
else:
  WORK_DIR = "/content/drive/MyDrive/ORT/Tesis/Codes/diffusion"

## HF token 

In [ ]:
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = ""
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

## Settings

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [ ]:
# MODEL_NAME = "runwayml/stable-diffusion-v1-5"

# weights model folder
OUTPUT_DIR = "/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion/models/stable-diffusion-v1-5-pneumoconiosis-finetuned"

print(f"[*] Weights will be load from {OUTPUT_DIR}")

[*] Weights will be load from /content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion/models/stable-diffusion-v1-5-pneumoconiosis-finetuned


In [ ]:
# Specify the weights directory to use (leave blank for latest)
WEIGHTS_DIR = "" 
if WEIGHTS_DIR == "":
    WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + sep + "*"))[-1]
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")

[*] WEIGHTS_DIR=/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion/models/stable-diffusion-v1-5-pneumoconiosis-finetuned/11400


In [ ]:
# folder to store the weights model for both classes finetuning
OUTPUT_BOTH_DIR = "/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion/models/stable-diffusion-v1-5-pneumoconiosis-finetuned-both"

print(f"[*] Weights will be saved at {OUTPUT_BOTH_DIR}")

[*] Weights will be saved at /content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion/models/stable-diffusion-v1-5-pneumoconiosis-finetuned-both


In [ ]:
# Specify the weights directory to use (leave blank for latest)
WEIGHTS_BOTH_DIR = "" 
if WEIGHTS_BOTH_DIR == "":
    WEIGHTS_BOTH_DIR = natsorted(glob(OUTPUT_BOTH_DIR + sep + "*"))[-1]
print(f"[*] WEIGHTS_DIR={WEIGHTS_BOTH_DIR}")

[*] WEIGHTS_DIR=/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion/models/stable-diffusion-v1-5-pneumoconiosis-finetuned-both/11400


In [ ]:
def generate_images(num_imgs: int, prompt: str, silent: bool=False) -> np.array: 
    negative_prompt = "" 
    guidance_scale = 7.5 
    num_inference_steps = 24 
    height = 512 
    width = 512 
    
 
    if silent:
        original = tqdm.__init__
        tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)
    
    with autocast(str(DEVICE)), torch.inference_mode():
        gen_imgs = pipe(
            prompt,
            height=height,
            width=width,
            negative_prompt=negative_prompt,
            num_images_per_prompt=num_imgs,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=g_cuda
        ).images
    
    if silent:
        tqdm.__init__ = original
        
    return gen_imgs

# Inference positive class model

In [ ]:
model_path = WEIGHTS_DIR

pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to(DEVICE)
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
g_cuda = ""

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
g_cuda = torch.Generator(device=DEVICE)
g_cuda.manual_seed(52362)

## Generate images

In [ ]:
IMAGE_FOLDER = WORK_DIR + '/' + DATA_DIR + '/pneumoconiosis' + '/diffusion' + '/StableDiffusionV1' + '/dreambooth_V1-5_finetuned'
if not exists(IMAGE_FOLDER):
    !mkdir "$IMAGE_FOLDER"
IMAGE_FOLDER

'/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion/../datasets/generated_datasets/pneumoconiosis/diffusion/StableDiffusionV1/dreambooth_V1-5_finetuned_test'

In [ ]:
num_images = 114
for i in tqdm(range(num_images), total=num_images):
    gen_image = generate_images(1, "image of a pneumoconiosis xray", silent=True)[0]
    gen_image = gen_image.convert('L')
    imageio.imwrite(join(IMAGE_FOLDER, f"{i}.png"), gen_image)

  0%|          | 0/114 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  1%|          | 1/114 [00:04<09:03,  4.81s/it]

  0%|          | 0/24 [00:00<?, ?it/s]

  2%|▏         | 2/114 [00:09<08:56,  4.79s/it]

...

 98%|█████████▊| 112/114 [09:08<00:09,  4.91s/it]

  0%|          | 0/24 [00:00<?, ?it/s]

 99%|█████████▉| 113/114 [09:13<00:04,  4.94s/it]

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 114/114 [09:18<00:00,  4.90s/it]


# Inference both class model

In [ ]:
model_both_path = WEIGHTS_BOTH_DIR

pipe = StableDiffusionPipeline.from_pretrained(model_both_path, safety_checker=None, torch_dtype=torch.float16).to(DEVICE)
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
g_cuda = ""

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
g_cuda = torch.Generator(device=DEVICE)
seed = 52362
g_cuda.manual_seed(seed)

## Generate images

In [ ]:
IMAGE_FOLDER = WORK_DIR + '/' + DATA_DIR + '/pneumoconiosis' + '/diffusion' + '/StableDiffusionV1' + '/dreambooth_V1-5_finetuned-both'
if not exists(IMAGE_FOLDER):
    !mkdir "$IMAGE_FOLDER"
IMAGE_FOLDER

'/content/drive/MyDrive/ORT/Tésis Máster/Codes/diffusion/../datasets/generated_datasets/pneumoconiosis/diffusion/StableDiffusionV1/dreambooth_V1-5_finetuned-both'

In [ ]:
num_images = 114
for i in tqdm(range(num_images), total=num_images):
    gen_image = generate_images(1, "image of a pneumoconiosis xray", silent=True)[0]
    imageio.imwrite(join(IMAGE_FOLDER, f"{i}.png"), gen_image)

  0%|          | 0/114 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  1%|          | 1/114 [00:12<23:37, 12.55s/it]

  0%|          | 0/24 [00:00<?, ?it/s]

  2%|▏         | 2/114 [00:17<14:35,  7.82s/it]

...

 98%|█████████▊| 112/114 [09:20<00:10,  5.00s/it]

  0%|          | 0/24 [00:00<?, ?it/s]

 99%|█████████▉| 113/114 [09:25<00:04,  4.99s/it]

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 114/114 [09:30<00:00,  5.00s/it]


In [ ]:
IMAGE_FOLDER = f"{IMAGE_FOLDER}_balanced"
if not exists(IMAGE_FOLDER):
    !mkdir "$IMAGE_FOLDER"

num_images = 339
for i in tqdm(range(num_images), total=num_images):
    gen_image = generate_images(1, "image of a pneumoconiosis xray", silent=True)[0]
    imageio.imwrite(join(IMAGE_FOLDER, f"{i}.png"), gen_image)

  0%|          | 0/339 [00:00<?, ?it/s]

In [ ]:
# Run Gradio UI for generating images.
# def inference(prompt, negative_prompt, num_samples, height=512, width=512, num_inference_steps=50, guidance_scale=7.5):
#     with torch.autocast("cuda"), torch.inference_mode():
#         return pipe(
#                 prompt, height=int(height), width=int(width),
#                 negative_prompt=negative_prompt,
#                 num_images_per_prompt=int(num_samples),
#                 num_inference_steps=int(num_inference_steps), guidance_scale=guidance_scale,
#                 generator=g_cuda
#             ).images

# with gr.Blocks() as demo:
#     with gr.Row():
#         with gr.Column():
#             prompt = gr.Textbox(label="Prompt", value="photo of zwx dog in a bucket")
#             negative_prompt = gr.Textbox(label="Negative Prompt", value="")
#             run = gr.Button(value="Generate")
#             with gr.Row():
#                 num_samples = gr.Number(label="Number of Samples", value=4)
#                 guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
#             with gr.Row():
#                 height = gr.Number(label="Height", value=512)
#                 width = gr.Number(label="Width", value=512)
#             num_inference_steps = gr.Slider(label="Steps", value=24)
#         with gr.Column():
#             gallery = gr.Gallery()

#     run.click(inference, inputs=[prompt, negative_prompt, num_samples, height, width, num_inference_steps, guidance_scale], outputs=gallery)

# demo.launch(debug=True)